In [17]:
from tkinter import *
from tkinter import filedialog
import fpdf
import cv2
import numpy as np
import pytesseract
from PIL import Image
from summarizer import Summarizer

pytesseract.pytesseract.tesseract_cmd = r"C:tesseract.exe"

# Path of working folder on Disk

def browseFiles():
    py=r"*.png *.jpg *jpeg"
    global result
    filename = filedialog.askopenfilename(initialdir = "/",title = "Select a File",filetypes = (("images",py),
                                                                                                ("all files","*.*")))
    if filename == "":
        return
    
    # Read image with opencv
    img = cv2.imread(filename)

    # Convert to gray
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply dilation and erosion to remove some noise
    kernel = np.ones((1, 1), np.uint8)
    img = cv2.dilate(img, kernel, iterations=1)
    img = cv2.erode(img, kernel, iterations=1)

    # Write image after removed noise
    cv2.imwrite("removed_noise.png", img)

    # Apply threshold to get binary image
    _, binary = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    
    cv2.imwrite("binary_image.png", binary)

    # Recognize text with tesseract for python
    result = pytesseract.image_to_string(Image.open("binary_image.png"))
    model = Summarizer()
    result= model(result)
    # Remove template file
    label_file_explorer.configure(text=result)
    
    

def pdf():
    global result
    pdf = fpdf.FPDF(format='letter')
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.write(5,result)
    pdf.ln()
    pdf.output("converted.pdf")

window = Tk()
  
# Set window title
window.title('File Explorer')
  
# Set window size
window.geometry("700x350")
reg_info = Label(window,text = "Text Recognition Using Pytesseract",width='80',height='2',font= ("ariel",12,"bold"),fg = "black",bg='lightgrey')
reg_info.place(x=370,y=18,anchor='center')  
#Set window background color
window.config(background = "white")
  
# Create a File Explorer label
label_file_explorer = Label(window,
                            text = "See the Output Here",font= ("ariel",10,"bold"),
                            width = 90, height = 12,
                            fg = "blue")
  
label_file_explorer.place(x=0,y=35) 

button_explore = Button(window,
                        text = "Browse Files",fg="white",bg="black",font= ("ariel",10,"bold"),width=10,
                        command = browseFiles)
button_explore.place(x=250,y=270)

text=Label(window,text="(Select an image)",bg="white",fg="black",font= ("ariel",8,"bold"))
text.place(x=242,y=300)

button1 = Button(window,
                        text = "convert text to pdf",fg="white",bg="black",font= ("ariel",10,"bold"),width=15,
                        command = pdf)
button1.place(x=370,y=270)

window.mainloop()

C:\Users\A******\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\A******\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


In [16]:
from tkinter import *
from tkinter import filedialog
import fpdf
import cv2
import numpy as np
import os
import pytesseract
from PIL import Image
from summarizer import Summarizer
import PyPDF2
from docx import Document
from pptx import Presentation

# Set Tesseract OCR path
pytesseract.pytesseract.tesseract_cmd = r"C:tesseract.exe"

def browseFiles():
    global result
    filename = filedialog.askopenfilename(initialdir="/", title="Select a File",
                                          filetypes=(("PDF files", "*.pdf"),
                                                     ("Word documents", "*.docx"),
                                                     ("PowerPoint presentations", "*.pptx"),
                                                     ("All files", "*.*")))
    if filename == "":
        return
    
    _, file_extension = os.path.splitext(filename)

    if file_extension.lower() == '.pdf':
        result = extract_text_from_pdf(filename)
    elif file_extension.lower() == '.docx':
        result = extract_text_from_docx(filename)
    elif file_extension.lower() == '.pptx':
        result = extract_text_from_pptx(filename)
    else:
        result = "Unsupported file format"

    model = Summarizer()
    result = model(result)
    print(result)
    label_file_explorer.configure(text=result)

def extract_text_from_pdf(filename):
    text = ""
    with open(filename, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        num_pages = len(pdf_reader.pages)
        for page_num in range(num_pages):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()
    return text

def extract_text_from_docx(filename):
    doc = Document(filename)
    text = ""
    for paragraph in doc.paragraphs:
        text += paragraph.text + '\n'
    return text

def extract_text_from_pptx(filename):
    prs = Presentation(filename)
    text = ""
    for slide in prs.slides:
        for shape in slide.shapes:
            if hasattr(shape, 'text'):
                text += shape.text + '\n'
    return text

def pdf():
    global result
    pdf = fpdf.FPDF(format='letter')
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.write(5,result)
    pdf.ln()
    pdf.output("converted.pdf")

window = Tk()
  
# Set window title
window.title('File Explorer')
  
# Set window size
window.geometry("1000x550")
reg_info = Label(window,text = "Handwritten Text Recognition Using Pytesseract",width='150',height='2',font= ("ariel",12,"bold"),fg = "black",bg='lightgrey')
reg_info.place(x=370,y=18,anchor='center')  
#Set window background color
window.config(background = "white")
  
# Create a File Explorer label
label_file_explorer = Label(window,
                            text = "See the Output Here",font= ("ariel",10,"bold"),
                            width = 150, height = 20,
                            fg = "blue")
  
label_file_explorer.place(x=0,y=35) 

button_explore = Button(window,
                        text = "Browse Files",fg="white",bg="black",font= ("ariel",10,"bold"),width=10,
                        command = browseFiles)
button_explore.place(x=250,y=270)

text=Label(window,text="(Select an file)",bg="white",fg="black",font= ("ariel",8,"bold"))
text.place(x=250,y=300)

button1 = Button(window,
                        text = "convert text to pdf",fg="white",bg="black",font= ("ariel",10,"bold"),width=15,
                        command = pdf)
button1.place(x=400,y=270)

window.mainloop()

In [10]:
files=["C:LS.pptx",
"C:doc.docx",
"resume.pdf"]

In [14]:
import PyPDF2
from docx import Document
from pptx import Presentation

def extract_text_from_pdf(file_path):
    text = ""
    with open(file_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        for page_num in range(len(pdf_reader.pages) ):
            page = pdf_reader.pages[page_num]
            text += page.extract_text ()
    return text

def extract_text_from_docx(file_path):
    text = ""
    doc = Document(file_path)
    for paragraph in doc.paragraphs:
        text += paragraph.text + '\n'
    return text

def extract_text_from_pptx(file_path):
    text = ""
    presentation = Presentation(file_path)
    for slide in presentation.slides:
        for shape in slide.shapes:
            if hasattr(shape, 'text'):
                text += shape.text + '\n'
    return text

def extract_text_based_on_extension(file_path):
    extension = file_path.split('.')[-1].lower()
    if extension == 'pdf':
        return extract_text_from_pdf(file_path)
    elif extension == 'docx':
        return extract_text_from_docx(file_path)
    elif extension == 'pptx':
        return extract_text_from_pptx(file_path)
    else:
        raise ValueError("Unsupported file type")

file_path = files[2]
extracted_text = extract_text_based_on_extension(file_path)
model = Summarizer()
result= model(extracted_text)
print(result)

******  
*******  *****  | http://www.linkedin.com/in/************** 
+91-***********  | ad***************@gmail.com   
As -----------------------------------------------------------------------------------------------------------------------, 
----------------------------------------------------------------- 
-----------------------------------------------------------------------------------------.
